# Regression with PyTorch on Alvis

In [ ]:
import torch
from torch import nn

## Constructing the data

In [ ]:
def f_true(x, slope=0.5, bias=0.3):
    '''The true underlying relation.'''
    return slope * x + bias

n_points = 300
noise_level = 0.1
x = 2 * torch.rand(n_points, 1) - 1
y = f_true(x) + noise_level * torch.randn(n_points, 1)

### Take a look at the data
As this is a notebook we can use the fact that we can easily take a look at graphical objects.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(x, y, '.', label="Data")
x_plot = torch.linspace(-1, 1, 20)
plt.plot(x_plot, f_true(x_plot), label="Noiseless relation")
plt.xlabel("X")
plt.ylabel("Y")

plt.legend();

## Constructing the model

In [ ]:
class LinearModel(nn.Module):
    '''A PyTorch linear regression model.'''
    def __init__(self, in_features, out_features):
        super().__init__()
        # In this function initialize objects that we want to use later
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        # Here we define the forward pass
        # PyTorch will keep track of the computational graph in the background,
        # which means we don't have to worry about implementing the backwards pass
        return self.linear(x)

# Note that in this simple case, we could have simply done
# model = nn.Linear(in_features=1, out_features=1)
# directly.
model = LinearModel(in_features=1, out_features=1)


## Training the model

In [ ]:
def train(model, loss_function, optimizer, n_epochs=20):
    '''Training the model.'''
    # Notify model to use training settings w.r.t. dropout etc.
    model.train()
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1:2d}/{n_epochs}", end="")
        
        # Reset optimizer
        optimizer.zero_grad()

        # Forward pass
        y_pred = model(x)
        loss = loss_function(y_pred, y)
        
        print(f"\tLoss {loss:g}")
        
        # Backward pass
        loss.backward()
        optimizer.step()

# Specify loss function and link optimizer with model parameters
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.3)

# Start the training
train(model, loss_function, optimizer)

## Evaluating the model

In [ ]:
def eval(model, metric):
    '''Evaluating the model'''
    model.eval()
    # We don't need to calculate any gradients
    with torch.no_grad():
        return metric(model(x), y)

loss = eval(model, loss_function)
print(f"Loss: {loss:g}")

### Visualising model predictions

In [ ]:
plt.figure()
plt.plot(x, y, '.', label="Data")
x_plot = torch.linspace(-1, 1, 20).unsqueeze(1)
plt.plot(x_plot, f_true(x_plot), label="Noiseless relation")
plt.xlabel("X")
plt.ylabel("Y")

# Add model prediction
model.eval()
with torch.no_grad():
    plt.plot(x_plot, model(x_plot), label="Predicted relation")

plt.legend();
